## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-08-20-54-50 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2026/01/08/world/...
1,2026-01-08-20-53-34 +0000,nyt,Live Updates: Senate Advances Measure to Curb ...,https://www.nytimes.com/live/2026/01/08/world/...
2,2026-01-08-20-51-52 +0000,nyt,"Trump ‘Always’ Respects Election Results, but ...",https://www.nytimes.com/2026/01/08/us/politics...
3,2026-01-08-20-49-14 +0000,nyt,Minnesota Officials Say They’re Being Blocked ...,https://www.nytimes.com/2026/01/08/us/minnesot...
4,2026-01-08-20-47-02 +0000,nyt,Australia Faces Critical Fire Warnings Amid Re...,https://www.nytimes.com/2026/01/08/weather/aus...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2365/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
8,trump,86
11,venezuela,37
24,ice,36
25,shooting,31
50,minneapolis,18


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
40,2026-01-08-20-10-00 +0000,wsj,President Trump lashed out at five GOP senator...,https://www.wsj.com/politics/policy/senate-vot...,192
141,2026-01-08-16-53-34 +0000,nyt,We Pressed Trump on His Conclusion About the I...,https://www.nytimes.com/2026/01/08/us/politics...,187
61,2026-01-08-19-28-52 +0000,nyt,Videos Contradict Trump Administration Account...,https://www.nytimes.com/video/us/1000000106310...,182
291,2026-01-08-06-43-54 +0000,nyt,Minnesota’s Dispute With Trump Administration ...,https://www.nytimes.com/2026/01/07/us/minnesot...,172
131,2026-01-08-17-23-16 +0000,nyt,"After ICE Shooting, Trump Administration Deplo...",https://www.nytimes.com/2026/01/08/us/politics...,172


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
40,192,2026-01-08-20-10-00 +0000,wsj,President Trump lashed out at five GOP senator...,https://www.wsj.com/politics/policy/senate-vot...
38,134,2026-01-08-20-12-04 +0000,nypost,Car attacks on ICE agents spiked by more than ...,https://nypost.com/2026/01/08/us-news/car-atta...
205,57,2026-01-08-13-00-00 +0000,wsj,The U.S. pumps more oil than any other country...,https://www.wsj.com/business/energy-oil/us-ven...
306,56,2026-01-08-04-25-00 +0000,wsj,"Rep. Steny Hoyer of Maryland, a longtime membe...",https://www.wsj.com/politics/policy/steny-hoye...
338,53,2026-01-08-01-34-00 +0000,wsj,Trump Administration Draws Up New Legal Justif...,https://www.wsj.com/politics/policy/trump-admi...
296,48,2026-01-08-06-05-45 +0000,latimes,'Public execution.' 'Trigger happy.' Californi...,https://www.latimes.com/california/story/2026-...
65,45,2026-01-08-19-14-24 +0000,nypost,Protests erupt in Iranian capital after exiled...,https://nypost.com/2026/01/08/world-news/prote...
215,41,2026-01-08-12-01-00 +0000,wsj,China Warns AI Startups Seeking to Emulate Met...,https://www.wsj.com/tech/china-warns-ai-startu...
175,40,2026-01-08-14-56-31 +0000,nypost,‘CBS Evening News’ No. 2 producer fired during...,https://nypost.com/2026/01/08/media/cbs-evenin...
104,34,2026-01-08-18-13-50 +0000,nypost,Russia rejects Trump’s 20-point peace plan for...,https://nypost.com/2026/01/08/us-news/russia-r...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
